This tutorial will introduce how to train a Gaussian Process Regression (GPR) model and evaulate its effectiveness for a materials or chemical property. GPR is a kernel based method that assumes each sample in a data set is drawn from a gaussian process (random variable) and uses that to generate a mean prediction across all space with an associated uncertainty. In essence what GPR is doing is fitting a basis-set expansion of a true function where the basis set is defined by the training data and a kernel, which define the location and shape of the basis functions, respectively. For small to medium sized datasets GPR is a powerful technique, but can become expensive for a larger number of data points (thousands) and features (hundreds).

This tutorial will demonstrate what GPRs are doing on a toy 1D problem, and demonstrate how to actually use it for a real material system by modelling the volume of a cubic-perovskite unitcell.

To start we will generate a simple example data set to demonstrate how GPR actually models a function, by create a test function with numpy

In [ ]:
import numpy as np

X = np.linspace(start=0, stop=10, num=1_000).reshape(-1, 1)
y = np.squeeze(X * np.sin(X))

Now that the function exists lets visualize it using matplotlib

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
plt.title("True generative process")
plt.show()

We can now randomly select some data points to act as a training set using a random number generator. 

These pseudo random number generators use a seed integer value to produce a series of numbers that follow a particular distribution to approximate randomness.

In [ ]:
rng = np.random.RandomState(1)
training_indices = rng.choice(np.arange(y.size), size=6, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

With this training set we can now train our first GPR model using scikit-learn, one of the more popular general-ML packages for python using the radial basis function (RBF) kernel. The RBF kernel is one of the more popular ones used in machine learning and is defined as a Gaussian distance matrix $K\left(\mathbf{x}, \mathbf{x'}\right)=\exp\left(\frac{\left|\left| \mathbf{x} - \mathbf{x'} \right|\right|^2}{2 \sigma^2}\right),$ with a hyperparmeter of $\sigma$ representing the length scale of the Gaussian. With scikit-learn we can set the bounds of the length scale and it will find the optimial value for training in that region.

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(X_train, y_train)
gaussian_process.kernel_

We can now use this model to get an estimate of the true function across the defined domain. Because we are using the exact data and did not set a regularization value, the model goes through each of the points exactly with no uncertainty. While it might be tempting to think of uncertainty as an error metric it is important to remember that uncertainty and error represent two different concepts. 

In [ ]:
mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.scatter(X_train, y_train, label="Observations")
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on noise-free dataset")

**Problem 1**

Select a different set of random points, retrain a GPR model, and describe what the differences are

In [ ]:
# For consistent results upon rerunning cells
rng = np.random.RandomState(5)

training_indices = rng.choice(np.arange(y.size), size=6, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(X_train, y_train)
gaussian_process.kernel_

mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.scatter(X_train, y_train, label="Observations")
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on noise-free dataset")

While the above models can accurately predict the function with a small number of data points, if they are somewhat evenly distributed, it was trained on perfect data (without noise or any errors). Unfortuately, in chemistry and materials science we almost never have perfect data. What happens if we add noise to the data and model?

In [ ]:
rng = np.random.RandomState(1)

training_indices = rng.choice(np.arange(y.size), size=6, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

# rng can also use other distributions than the standard uniform one used normally
noise_std = 0.75
y_train_noisy = y_train + rng.normal(loc=0.0, scale=noise_std, size=y_train.shape)

We can now add a term to our GPR model to account for this additional uncertainty of our data set $\alpha$ represents the intrinsic variance in our measurements and can be set to the square of the noise's standard devation. To account for this we can also restart the training procedure to better estimate the kernel's hyperparameters. 

In [ ]:
gaussian_process = GaussianProcessRegressor(
    kernel=kernel, alpha=noise_std**2, n_restarts_optimizer=9
)
gaussian_process.fit(X_train, y_train_noisy)
mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)
print(gaussian_process.kernel_)

We can now plot the results and compare it to the previous exmples. Because of the $\alpha$ parameter there is never a point where the uncertainty goes to zero, and overall the model is less confident of what the true function looks like

In [ ]:
plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.errorbar(
    X_train,
    y_train_noisy,
    noise_std,
    linestyle="None",
    color="tab:blue",
    marker=".",
    markersize=10,
    label="Observations",
)
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    color="tab:orange",
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on a noisy dataset")

**Problem 2**

Incrase the std_noise to 1.5 and reduce it 0.375 and see how that affects the model performance

In [ ]:
noise_std = 1.5
y_train_noisy = y_train + rng.normal(loc=0.0, scale=noise_std, size=y_train.shape)

gaussian_process = GaussianProcessRegressor(
    kernel=kernel, alpha=noise_std**2, n_restarts_optimizer=9
)
gaussian_process.fit(X_train, y_train_noisy)
mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)
print(gaussian_process.kernel_)

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.errorbar(
    X_train,
    y_train_noisy,
    noise_std,
    linestyle="None",
    color="tab:blue",
    marker=".",
    markersize=10,
    label="Observations",
)
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    color="tab:orange",
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on a noisy dataset")

In [ ]:
noise_std = 0.375
y_train_noisy = y_train + rng.normal(loc=0.0, scale=noise_std, size=y_train.shape)

gaussian_process = GaussianProcessRegressor(
    kernel=kernel, alpha=noise_std**2, n_restarts_optimizer=9
)
gaussian_process.fit(X_train, y_train_noisy)
mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)
print(gaussian_process.kernel_)

plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.errorbar(
    X_train,
    y_train_noisy,
    noise_std,
    linestyle="None",
    color="tab:blue",
    marker=".",
    markersize=10,
    label="Observations",
)
plt.plot(X, mean_prediction, label="Mean prediction")
plt.fill_between(
    X.ravel(),
    mean_prediction - 1.96 * std_prediction,
    mean_prediction + 1.96 * std_prediction,
    color="tab:orange",
    alpha=0.5,
    label=r"95% confidence interval",
)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Gaussian process regression on a noisy dataset")

Now that we have an understanding of what GPR is doing, now lets apply it to a real-world problem for predicting the volume of a cubic-perovskite unit cell volume. 

For this problem we will take a data set of preovskite's volumes and the average value for a series of average elemental properties of the given atoms inside of that unit cell taken from online databases, you can see more information about this here: [Journal of Materials Research 38 (20), 4477-4496](https://link.springer.com/article/10.1557/s43578-023-01164-w).

To get the data we are using the pandas DataFrame and reading from a CSV file. In the ML community and statistics in general pandas is one of the most widely used packages with support for numpy, matplotlib, and scikit-learn. Here we are going to pull out the volume information and use that as our y-points or our target property

In [ ]:
import pandas as pd
df = pd.read_csv("perovskite_data.csv", index_col=0)

volume = df["Volume"]
df.drop(columns=["Volume"], inplace=True)

for col in df.columns:
    print(col)

From this data we can simply train a model with an estimate (guess) of what the variance term should be and plot the results to see a near-perfect agrreement with the calucated values.

These types of plots are called parity plots and are a commonly used tool for seeing how well a model fits the data. The closer all points are to the diagonal the better.

In [ ]:
gaussian_process = GaussianProcessRegressor(
    kernel=kernel, alpha=1.0, n_restarts_optimizer=9
)

gaussian_process.fit(df, volume)
vol_prediction, vol_std = gaussian_process.predict(df, return_std=True)

plt.plot(vol_prediction, volume, 'o')
plt.ylabel("Calculated Volume ($\\AA^3$)")
plt.xlabel("Predicted Volume ($\\AA^3$)")
plt.show()

This result looks quite impressive, but in reality it is meaningless. The training error can approach 0 by simply fitting a linear model with $n$ random vectors, where $n$ is the number of data points in the training set. This model will perfectly fit all data in the training set, but not be able to predict any new point.

To evaluate how well a model is doing we can perform a cross-validation where we split the data set into a training and validation sets and evaulate a model trained on a training set with the validation set. Here we will use 5-fold cross validation which splits the dataset into 5 training and validation sets where each data point in the dataset appears in exactly one validation set.

While this will give a decent estimate of the "true" model error, the best way to do this is still an active area of research.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

k_fold = KFold(n_splits=5)
vol_pred = np.zeros(len(volume))
rmses = []
for train_inds, test_inds in k_fold.split(df):
    print(f"Test inds: {test_inds}")
    
    X_train = df.iloc[train_inds, :]
    X_test = df.iloc[test_inds, :]
    
    y_train = volume.values[train_inds]
    y_test = volume.values[test_inds]

    gaussian_process.fit(X_train, y_train)
    vol_pred[test_inds] = gaussian_process.predict(X_test)
    
    rmses.append(np.sqrt(np.mean((vol_pred[test_inds] - y_test) ** 2.0)))

print(rmses)
plt.plot(vol_pred, volume, 'o')
plt.plot([10, 300], [10, 300])
plt.ylabel("Calculated Volume ($\\AA^3$)")
plt.xlabel("Predicted Volume ($\\AA^3$)")
plt.show()

We now see that our model is not really doing well with it having a large tendency to under-estimate the actually calculated values. This means that our model is over-fitting and tailoring the model to match the dataset. This is a common thing in machine learning, which is why cross-validation is paramount to do for any ML project.

Now we want to see if we can improve this model by tuning the hyperparameters for the GPR model: The alpha parameter and the length scale of the kernel. This procedure is known as hyperparameter optimization and is setting the free parameters of a ML model that are not directly set from the training process. In the above example we made a guess of what they should be, and unsurprisingly that guess was bad, a systematic search for the optimal values must be done. 

There are multiple ways of tuning these parameters, but we will use the simplest method, a basic grid search. This is going to basically try a range of values to see what performs best, but other methods do this more efficently. 

1) Define Functions needed for cross-validation

In [ ]:
def train_model(X, y, alpha, length_scale):
    """Train a gaussian process regression model

    Args:
        X (pd.DataFrame): Training data
        y (np.array[float]): Training labels
        alpha (float): Gaussian Measurement noise
        length_scale (float): Length scale of the kernel

    Returns:
        GaussianProcessRegressor: Trained model
    """
    kernel = 1 * RBF(length_scale=length_scale, length_scale_bounds=(1e-12, 1e12))
    
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=alpha)
    gpr.fit(X, y)
    
    return gpr

def k_fold_cross_valiation(X, y, n_splits, alpha, ls):
    """Perform a 5-fold cross-validation on a data-set X, y

    Args:
        X (pd.DataFrame): The training data
        y (pd.Series): The training labels
        n_splits (int): The number of splits to use
        alpha (float): The alpha parameter for GPR
        ls (float): Length scale for the RBF Kernel
    
    Returns:
        tupe[np.ndarray[float], np.ndarray[float]]: The mean prediction and the std of the predictions
    """    
    k_fold = KFold(n_splits=n_splits)

    y_pred = np.zeros(len(y))
    y_std = np.zeros(len(y))
    for train_inds, test_inds in k_fold.split(X):
        gpr = train_model(
            X.iloc[train_inds, :], 
            y.iloc[train_inds],
            alpha,
            ls,
        )
        y_pred[test_inds], y_std[test_inds] = gpr.predict(X.iloc[test_inds, :], return_std=True)

    return y_pred, y_std

def rmse(y_true, y_pred):
    """Caclulate the rmse of an approximation
    Args:
        y_true (np.array[float]): The True values
        y_pred (np.array[float]): The predicted values

    Returns:
        float: The RMSE of the model
    """
    return np.sqrt(np.mean((y_true - y_pred) ** 2.0))


2) Define grid of hyperparameters

In [ ]:
alphas = 10.0 ** np.arange(-6, 3, 1)
length_scale = 10.0 ** np.arange(-4, 4, 1)

Initialize The min values of interest

In [ ]:
min_rmse = np.inf # Min RMSE should be the highest possible value (np.inf, infinity)
min_hp = ()
min_preds = []
min_stds = []

Loop over all combinations of alpha and ls in a grid search using iter tools product

Itertools prodcut takes two lists and creates all combinations: Example

```
lst_1 = [A, B, C, D]
lst_2 = [1, 2]
product(lst_1, lst_2) = [(A, 1), (A, 2), (B, 1), (B, 2), (C, 1), (C, 2), (D, 1), (D, 2)]
```

In [ ]:
from itertools import product

for alpha, ls in product(alphas, length_scale):
    vol_pred, vol_std = k_fold_cross_valiation(df, volume, 5, alpha, ls)
    
    rmse_cur = rmse(vol_pred, volume.values)
    if rmse_cur < min_rmse:
        print(f"alpha: {alpha}, ls: {ls}, rmse: {rmse_cur}, ")
        min_rmse = rmse_cur
        min_hp = (alpha, ls)
        min_preds = vol_pred
        min_std = vol_std

We can now see how well this model is performing by plotting the validation error from the cross-validation examples.

To make it easier lets define a plotting function

In [ ]:
from scipy.stats import pearsonr

def plot_parity_plot(y_true, y_pred, y_std):
    """Plots a parity plot of a model

    Args:
        y_true (np.ndarray[float]): The True values of the model
        y_pred (np.ndarray[float]): The predicted values
        y_std (np.array[float]): The standard deviation of the model

    Returns:
        mpl.Figure, mpl.Axes: The figure and axes of the parity plot
    """
    R = pearsonr(y_true, y_pred).statistic
    rmse_val = rmse(y_true, y_pred)
    
    min_val = min(np.min(y_true), np.min(y_pred))
    max_val = max(np.max(y_true), np.max(y_pred))
    
    fig, ax = plt.subplots()
    # Plot the parity line as a gray dashed line
    ax.plot([min_val - 10, max_val + 10], [min_val - 10, max_val + 10], "--", c="gray")
    plt.errorbar(
        y_true,
        y_pred,
        y_std,
        linestyle="None",
        color="tab:blue",
        marker=".",
        markersize=10,
        label="Observations",
    )
    ax.set_xlabel("Calculated Volume ($\\AA^3$)")
    ax.set_ylabel("Predicted Volume ($\\AA^3$)")
    
    ax.set_xlim([0, 310])
    ax.set_ylim([0, 310])
    
    ax.text(10, 295, f"Validation RMSE: {rmse_val:0.1f}")
    ax.text(10, 275, f"Validation R$^2$: {R ** 2.0:0.2f}")

    return fig, ax

In [ ]:
fig, ax = plot_parity_plot(volume, min_preds, min_std)

**Problem 3**

Create a hyperparameter optimization function for alpha and ls and use it to see how changing the number of splits affects the validation results.

Things to think about

1) What are our inputs?
2) What is the output?
3) What code should we use?

In [ ]:
def hyper_param_opt(X, y, n_splits, alphas=None, lengh_scales=None):
    """Optimize the hpyerparameters for a given dataset

    Args:
        X (pd.DataFrame): training data
        y (pd.Series): Training labels
        n_splits (int): number of splits for k-fold cross_validation
        alphas (Optional[np.ndarray[float]]): List of all alpha values to test
        lengh_scales (Optional[np.ndarray[float]]): List of all length_scale values to test

    Returns:
        tuple[int, int]: The optimized alpha and length scale
    """
    if alphas is None:
        alphas = 10.0 ** np.arange(-4, 3, 1)
    
    if lengh_scales is None:
        length_scales = 10.0 ** np.arange(-4, 3, 1)

    min_hp = ()
    min_rmse = 1e10
    
    for alpha, ls in product(alphas, length_scales):
        y_pred, _ = k_fold_cross_valiation(X, y, n_splits, alpha, ls)
        rmse_cur = rmse(y.values, y_pred)
        if rmse_cur < min_rmse:
            min_rmse = rmse_cur
            min_hp = (alpha, ls)
    return min_hp

alpha, ls = hyper_param_opt(df, volume, 2)
vol_pred, vol_std = k_fold_cross_valiation(df, volume, 2, alpha, ls)
fig_2, ax_2 = plot_parity_plot(volume, vol_pred, vol_std)
ax_2.text(10, 255, f"n_splits: 2")

alpha, ls = hyper_param_opt(df, volume, 10)
vol_pred, vol_std = k_fold_cross_valiation(df, volume, 10, alpha, ls)
fig_10, ax_10 = plot_parity_plot(volume, vol_pred, vol_std)
ax_10.text(10, 255, f"n_splits: 10")


As we can see from the answers of the last problem, in general the smaller your training set is relative to the overall dataset size, the larger the validation errors are for the trained models. We can check how our models are doing by generating the learning curves and comparing the training and test RMSEs with different sizes of our training data. Please note that better hyper-parameter optimization would be need to fully trust these results.

In [ ]:
rng = np.random.RandomState(1)
    
indexes = np.arange(len(volume), dtype=np.int64)
rng.shuffle(indexes)

X_shuffle = df.iloc[indexes, :]
y_shuffle = volume.iloc[indexes]

n_update = 15
n_train = n_update

train_rmse = []
test_rmse = []

while n_train < len(indexes):
    X_train = X_shuffle.iloc[:n_train, :]
    X_test = X_shuffle.iloc[n_train:, :]

    y_train = y_shuffle.iloc[:n_train]
    y_test = y_shuffle.iloc[n_train:]
    
    alpha, ls = hyper_param_opt(X_train, y_train, min(n_update, 5))
    print(n_train, alpha, ls)
    gpr = train_model(X_train, y_train, alpha, ls)
    
    train_rmse.append(rmse(y_train, gpr.predict(X_train)))
    test_rmse.append(rmse(y_test, gpr.predict(X_test)))
    
    n_train += n_update

plt.plot(range(n_update, len(indexes), n_update), train_rmse)
plt.plot(range(n_update, len(indexes), n_update), test_rmse)
plt.show()